In [1]:
%matplotlib notebook
from scipy.ndimage import sobel, laplace
from scipy.stats import wilcoxon
import numpy as np
import pickle
import matplotlib.pyplot as plt


def rescale_values(image,max_val,min_val):
    '''
    image - numpy array
    max_val/min_val - float
    '''
    return (image-image.min())/(image.max()-image.min())*(max_val-min_val)+min_val

SEED=1234
np.random.seed(SEED)
# torch.manual_seed(SEED)

In [2]:
# Scale matrix to sum to 1
def sum_to_1(mat):
    return mat / np.sum(mat)

def rand_baseline(data: np.array):
    rands = np.random.uniform(low=-1.0, high=1.0, size=(data.shape))
    return sum_to_1(rands)

def x_baseline(data: np.array):
    rgb_weights = [0.2989, 0.5870, 0.1140]
    greyscale = np.dot(data[...,:3], rgb_weights)
    return sum_to_1(greyscale)

In [3]:
# # Confounder Data
# with open('confounder_train128.pkl', 'rb') as f:
#     confounder_train = pickle.load(f)
#     confounder_train = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in confounder_train]

# with open('confounder_val128.pkl', 'rb') as f:
#     confounder_val = pickle.load(f)
#     confounder_val = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in confounder_val]

# with open('confounder_test128.pkl', 'rb') as f:
#     confounder_test = pickle.load(f)
#     confounder_test = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in confounder_test]

    
# # Suppressor Data
# with open('suppressor_train128.pkl', 'rb') as f:
#     supressor_train = pickle.load(f)
#     supressor_train = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in supressor_train]

# with open('suppressor_validation128.pkl', 'rb') as f:
#     supressor_val = pickle.load(f)
#     supressor_val = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in supressor_val]

# with open('suppressor_test128.pkl', 'rb') as f:
#     supressor_test = pickle.load(f)
#     supressor_test = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in supressor_test]


# # No Watermark Data
# with open('no_mark_train128.pkl', 'rb') as f:
#     no_mark_train = pickle.load(f)
#     no_mark_train = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in no_mark_train]

# with open('no_mark_validation128.pkl', 'rb') as f:
#     no_mark_val = pickle.load(f)
#     no_mark_val = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in no_mark_val]

# with open('no_mark_test128.pkl', 'rb') as f:
#     no_mark_test = pickle.load(f)
#     no_mark_test = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in no_mark_test]

In [4]:
with open('mark_all128.pkl', 'rb') as f:
    watermark_dataset = pickle.load(f)
    watermark_dataset = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in watermark_dataset]

with open('no_mark_test128.pkl', 'rb') as f:
    no_watermark_dataset = pickle.load(f)
    no_watermark_dataset = [[rescale_values(i[0],1,0).transpose(2,0,1),i[1]] for i in no_watermark_dataset]

In [5]:
train_energy=[]
methods = ['deconv', 'int_grads', 'shap', 'lrp', 'lrp_ab', 'laplace', 'sobel', 'x'] # lime just nan values
# methods = ['deconv', 'int_grads', 'shap', 'lrp']
n_test = 3000

energies = {}

for method in methods:
    comb_water_conf = []
    comb_water_sup = []
    comb_water_no = []
    
    comb_no_water_conf = []
    comb_no_water_sup = []
    comb_no_water_no = []

    # if method == 'lrp_ab':
    #     file_suffix = 'lrp_'
    # else:
    file_suffix=''

    energies[method] = {}
    
    for split in range(5):
        # for model_ind in range(10):
        file = open(f'./energies/energy_water_conf_pred_{file_suffix}{split}.pickle', 'rb')
        energy_water_conf = pickle.load(file)

        file = open(f'./energies/energy_water_sup_pred_{file_suffix}{split}.pickle', 'rb')
        energy_water_sup = pickle.load(file)

        file = open(f'./energies/energy_water_no_pred_{file_suffix}{split}.pickle', 'rb')
        energy_water_no = pickle.load(file)

        file = open(f'./energies/energy_no_water_conf_pred_{file_suffix}{split}.pickle', 'rb')
        energy_no_water_conf = pickle.load(file)

        file = open(f'./energies/energy_no_water_sup_pred_{file_suffix}{split}.pickle', 'rb')
        energy_no_water_sup = pickle.load(file)

        file = open(f'./energies/energy_no_water_no_pred_{file_suffix}{split}.pickle', 'rb')
        energy_no_water_no = pickle.load(file)
        
        comb_water_conf.extend(energy_water_conf[method][:n_test])
        comb_water_sup.extend(energy_water_sup[method][:n_test])
        # comb_water_no.extend(energy_water_no[method][:n_test])

        if method == 'laplace' or method == 'sobel' or method == 'x':
            comb_water_no.extend(energy_water_sup[method][:n_test])
            comb_no_water_conf.extend(energy_no_water_sup[method][:n_test])
        else:
            comb_water_no.extend(energy_water_no[method][:n_test])
            comb_no_water_conf.extend(energy_no_water_conf[method][:n_test])

        
        # comb_no_water_conf.extend(energy_no_water_conf[method][:n_test])
        comb_no_water_sup.extend(energy_no_water_sup[method][:n_test])
        comb_no_water_no.extend(energy_no_water_no[method][:n_test])

    energies[method]['conf'] = [comb_no_water_conf, comb_water_conf]
    energies[method]['sup'] = [comb_no_water_sup, comb_water_sup]
    energies[method]['no'] = [comb_no_water_no, comb_water_no]

    train_energy.append([[comb_no_water_conf, comb_water_conf],
                [comb_no_water_sup, comb_water_sup],
                [comb_no_water_no, comb_water_no]])

In [6]:
# energy_lapl_no = []
# energy_lapl_wm = []

# energy_sob_no = []
# energy_sob_wm = []

# energy_x_no = []
# energy_x_wm = []

# for split in range(10):
#     with open(f'./energies/energy_baselines_{split}.pickle', 'rb') as f:
#         [[lapl_no, lapl_wm], [sob_no, sob_wm], [x_no, x_wm]] = pickle.load(f)   

#     energy_lapl_no.extend(lapl_no)
#     energy_lapl_wm.extend(lapl_wm)
    
#     energy_sob_no.extend(sob_no)
#     energy_sob_wm.extend(sob_wm)
    
#     energy_x_no.extend(x_no)
#     energy_x_wm.extend(x_wm)


# baselines = {
#     'lapl': [energy_lapl_no, energy_lapl_wm],
#     'sob': [energy_sob_no, energy_sob_wm],
#     'x': [energy_x_no, energy_x_wm]
# }


# train_energy.append([ [energy_lapl_no, energy_lapl_wm], [energy_lapl_no, energy_lapl_wm], [energy_lapl_no, energy_lapl_wm] ])
# train_energy.append([ [energy_sob_no, energy_sob_wm], [energy_sob_no, energy_sob_wm], [energy_sob_no, energy_sob_wm] ])
# train_energy.append([ [energy_x_no, energy_x_wm], [energy_x_no, energy_x_wm], [energy_x_no, energy_x_wm] ])


# for baseline, results in baselines.items():
#     energies[baseline] = {}
#     for dataset in ['conf', 'sup', 'no']:
#         energies[baseline][dataset] = results
# # energies['lapl']['conf'] = [comb_no_water_conf, comb_water_conf]
# # energies[method]['sup'] = [comb_no_water_sup, comb_water_sup]
# # energies[method]['no'] = [comb_no_water_no, comb_water_no]

## Hypothesis 1
- if the XAI methods work as expected as confounder detectors, the relative energy should differ betwen W and NO-W test images only for the confounder case, but not for the other two.
- Using a [paired Wilcoxon signed-rank test](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) to test significance
- Evaluate differences between E(W_i) and E(NoW_i) for all three models for each given XAI method

In [7]:
def format_e(n):
    a = '%E' % n
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]

In [8]:
for method, results in energies.items():
    for model, result in results.items():
        res = wilcoxon(result[1], result[0], alternative='greater')
        print(method, model, format_e(res.statistic), res.pvalue)

deconv conf 1.018143E+08 0.0
deconv sup 8.200056E+07 0.0
deconv no 1.060115E+08 0.0
int_grads conf 1.073019E+08 0.0
int_grads sup 1.074785E+08 0.0
int_grads no 1.073915E+08 0.0
shap conf 1.073156E+08 0.0
shap sup 1.070529E+08 0.0
shap no 1.074322E+08 0.0
lrp conf 1.07463E+08 0.0
lrp sup 1.074079E+08 0.0
lrp no 1.076062E+08 0.0
lrp_ab conf 1.067605E+08 0.0
lrp_ab sup 1.060277E+08 0.0
lrp_ab no 1.063642E+08 0.0
laplace conf 1.122766E+08 0.0
laplace sup 1.122766E+08 0.0
laplace no 1.122766E+08 0.0
sobel conf 1.121219E+08 0.0
sobel sup 1.121219E+08 0.0
sobel no 1.121219E+08 0.0
x conf 5.526684E+07 0.0
x sup 5.526684E+07 0.0
x no 5.526684E+07 0.0


In [9]:
# Effect size as rank-biserial correlation
# aka (test statistic / total rank sum)
for method, results in energies.items():
    for model, result in results.items():
        res = wilcoxon(result[1], result[0], alternative='greater')
        rank_sum = sum(np.where(np.array(result[1]) ==  np.array(result[1]))[0])
        print(method, model, res.statistic/rank_sum, res.pvalue)

deconv conf 0.905076089517079 0.0
deconv sup 0.7289424628308554 0.0
deconv no 0.9423874924994999 0.0
int_grads conf 0.9538578838589239 0.0
int_grads sup 0.9554282196590884 0.0
int_grads no 0.9546550525590595 0.0
shap conf 0.953979900882281 0.0
shap sup 0.9516443762917528 0.0
shap no 0.9550169566860013 0.0
lrp conf 0.9552901571215859 0.0
lrp sup 0.9548009689534858 0.0
lrp no 0.9565631042069471 0.0
lrp_ab conf 0.9490457674956109 0.0
lrp_ab sup 0.9425313243105096 0.0
lrp_ab no 0.9455223859368402 0.0
laplace conf 0.998081116518879 0.0
laplace sup 0.998081116518879 0.0
laplace no 0.998081116518879 0.0
sobel conf 0.996705149232171 0.0
sobel sup 0.996705149232171 0.0
sobel no 0.996705149232171 0.0
x conf 0.4912935617930084 0.0
x sup 0.4912935617930084 0.0
x no 0.4912935617930084 0.0


In [29]:
test_result = energies['deconv']['conf']

res = wilcoxon(test_result[1], test_result[0], alternative='greater')
print(res)

WilcoxonResult(statistic=101814272.0, pvalue=0.0)


In [10]:
# import pickle as pkl

# with open('energy_laplace.pickle', 'rb') as f:
#     lapl = pkl.load(f)
    
# with open('energy_sobel.pickle', 'rb') as f:
#     sob= pkl.load(f)

In [11]:
# lapl_no, lapl_wm = lapl
# sob_no, sob_wm = sob

## Hypothesis 2
if the XAI methods work as expected as confounder detectors, then the relative energy on W test images should be higher for confounded training data then for the other two. If this is the case - how big is the effect?
- Meaning, how much more watermark energy is attributed by models trained on confounded data compared to models trained on unconfounded data.
- Calculate E(Wˆtr=Conf_i) - E(Wˆtr=Supp_i) and E(Wˆtr=Conf_i) - E(Wˆtr=NoW_i) for all XAI methods

In [12]:
for method, results in energies.items():
    #  E(Wˆtr=Conf_i) - E(Wˆtr=Supp_i) 
    try:
        res_sup = wilcoxon(results['conf'][1], results['sup'][1], alternative='greater')
        
        #  E(Wˆtr=Conf_i) - E(Wˆtr=NoW_i) 
        res_no = wilcoxon(results['conf'][1], results['no'][1], alternative='greater')

        res_no_2 = wilcoxon(results['no'][1], results['conf'][1], alternative='greater')
        
        rank_sum = sum(np.where(np.array(results['conf'][1]) ==  np.array(results['conf'][1]))[0])

        print(method, 'CONF - SUPP', res_sup.statistic/rank_sum, res_sup.pvalue)
        print(method, 'CONF - NO', res_no.statistic/rank_sum, res_no.pvalue)
        print(method, 'NO - CONF', res_no_2.statistic/rank_sum, res_no_2.pvalue)
    except:
        print(method, 'CONF - SUPP')
        print(method, 'CONF - NO')

deconv CONF - SUPP 0.8976305709269506 0.0
deconv CONF - NO 0.13096247305375913 1.0
deconv NO - CONF 0.8691708691690557 0.0
int_grads CONF - SUPP 0.8824618530124231 0.0
int_grads CONF - NO 0.02623913594239616 1.0
int_grads NO - CONF 0.9738942062804187 0.0
shap CONF - SUPP 0.8594591639442629 0.0
shap CONF - NO 0.043270715825499476 1.0
shap NO - CONF 0.9568626263973153 0.0
lrp CONF - SUPP 0.8403469742427273 0.0
lrp CONF - NO 0.1529426761784119 1.0
lrp NO - CONF 0.847190666044403 0.0
lrp_ab CONF - SUPP 0.8191147943196213 0.0
lrp_ab CONF - NO 0.4580823254883659 1.0
lrp_ab NO - CONF 0.542051016734449 2.664839004392391e-19
laplace CONF - SUPP
laplace CONF - NO
sobel CONF - SUPP
sobel CONF - NO
x CONF - SUPP
x CONF - NO


## Coloured MNIST

In [13]:
# method = 'shap'

# conf_conf = [[],[],[]]
# conf_sup = [[],[],[]]
# conf_no = [[],[],[]]

# sup_conf = [[],[],[]]
# sup_sup = [[],[],[]]
# sup_no = [[],[],[]]

# no_conf = [[],[],[]]
# no_sup = [[],[],[]]
# no_no = [[],[],[]]

# for model_ind in range(5):
#     file = open(f'energies_mnist_{model_ind}.pickle', 'rb')
#     energies = pickle.load(file)
    
#     for i in range(3):
#         conf_conf[i].extend(energies['conf']['conf'][i][method])
#         conf_sup[i].extend(energies['conf']['sup'][i][method])
#         conf_no[i].extend(energies['conf']['no_col'][i][method])
        
#         sup_conf[i].extend(energies['sup']['conf'][i][method])
#         sup_sup[i].extend(energies['sup']['sup'][i][method])
#         sup_no[i].extend(energies['sup']['no_col'][i][method])

#         no_conf[i].extend(energies['no_col']['conf'][i][method])
#         no_sup[i].extend(energies['no_col']['sup'][i][method])
#         no_no[i].extend(energies['no_col']['no_col'][i][method])
        
        
# combined_energies = {
#     'conf': { # conf model on conf/sup/no data
#         'conf': conf_conf,
#         'sup': conf_sup, 
#         'no_col': conf_no},
            
#     'sup': { # sup model on conf/sup/no data
#         'conf': sup_conf,
#         'sup': sup_sup, 
#         'no_col': sup_no},

#     'no_col': { # no_col model on conf/sup/no data
#         'conf': no_conf,
#         'sup': no_sup, 
#         'no_col': no_no},
# }

In [14]:
# wilcoxon(x=energies['lrp']['conf'][1], y=energies['lrp']['conf'][0], alternative='greater')